In [14]:
import os
import pandas as pd
import pycep_correios
from geopy.geocoders import Nominatim, HereV7, TomTom
from geopy.extra.rate_limiter import RateLimiter

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

uf = 'AL'
file_path = os.path.join(f'postos_saude_brasil_{uf}.csv')
df = pd.read_csv(file_path)

geolocator_nominatim = Nominatim(user_agent="info_vacinas")
geolocator_tomtom = TomTom(api_key='wvuAFmBut64DLjQqey19XtfuMXZilzbj', user_agent="info_vacinas")
geolocator_here = HereV7(apikey=os.environ.get('HERE_API_KEY'), user_agent="info_vacinas")
geocode_nominatim = RateLimiter(geolocator_nominatim.geocode, min_delay_seconds=0)
geocode_tomtom = RateLimiter(geolocator_tomtom.geocode, min_delay_seconds=0)
geocode_here = RateLimiter(geolocator_here.geocode, min_delay_seconds=0)
tamanho = len(df)

print(f'Processando {str(tamanho).zfill(5)} registros')
#for index, row in df[::-1].iterrows():
for index, row in df.iterrows():
  # pula os registros que já possuem dados de localização
  if row['latitude'] != None:
    print('Pulando, informações já preenchidas', row['latitude'])
    continue
  
  print(f'Processando {str(index+1).zfill(5)} de {str(tamanho).zfill(5)}')  
  
  # inicializa a variável que receberá o retorno das APIs de localização
  location = None

  try:
    # tenta fazer a localização usando o Nominatim
    location = geolocator_nominatim.geocode(row['ENDERECO_COMPLETO'])
  except Exception as e:
    print(f'Erro: {e}')
    # se a localização não retorna nada, tenta pegar a localização no TomTom
    try:
      location = geolocator_tomtom.geocode(row['ENDERECO_COMPLETO'])
    except Exception as e:
      # se a localização não retorna nada, tenta pegar a localização no Here
      print(f'Erro: {e}')
      try:
        location = geolocator_here.geocode(row['ENDERECO_COMPLETO'])
      except Exception as e:
        # salva o CSV com os dados buscados até aquele momento
        df.to_csv(file_path, index=False)
        if not location:
          continue

  # se em nenhuma das três tentativas conseguiu retornar a localização, 
  # vai para o próximo registro
  if not location:
    continue

  print(location)

  # salva de 10 em 10 registros o arquivo de saída
  if index%10 == 0:
    df.to_csv(file_path, index=False)

  row['latitude'] = location.latitude
  row['longitude'] = location.longitude
  row['address'] = location.address
  row['point'] = location.point

df.to_csv(file_path, index=False)
print('Arquivo de saída atualizado com sucesso.', file_path)

Processando 00243 registros
Pulando, informações já preenchidas -10.075610000000001
Pulando, informações já preenchidas -22.28388
Pulando, informações já preenchidas -10.076839999999999
Pulando, informações já preenchidas -17.34786
Pulando, informações já preenchidas -22.28064
Pulando, informações já preenchidas -10.076839999999999
Pulando, informações já preenchidas -10.9418
Pulando, informações já preenchidas -22.90713
Pulando, informações já preenchidas -10.938410000000001
Pulando, informações já preenchidas -10.94019
Pulando, informações já preenchidas -10.9991
Pulando, informações já preenchidas -10.990711099999999
Pulando, informações já preenchidas -10.93564
Pulando, informações já preenchidas -11.01544
Pulando, informações já preenchidas -11.01004
Pulando, informações já preenchidas -19.57619
Pulando, informações já preenchidas -10.93564
Pulando, informações já preenchidas -11.012260000000001
Pulando, informações já preenchidas -22.935879999999997
Pulando, informações já preenc